In [ ]:
#와인데이터 로드
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [ ]:
#입력데이터와 타깃 정의
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
#훈련데이터와 테스트데이터 정의
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [ ]:
#훈련데이터를 훈련데이터와 검증데이터로 나눔
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
#결정트리 모델 생성 및 검증데이터로 모델평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [ ]:
#교차검증 - k-fold cross validate
#교차검증함수: cross_validate
# - 매개변수: 첫 번째 인자(모델 객체), 두 번째 인자(훈련데이터 전체), 세 번째 인자(타깃)
# - 리턴값: 훈련시간, 검증시간, 검증점수를 원소로 하는 딕셔너리, 기본값으로 5-폴드 교차검증 수행, cv 매개변수로 k-폴드의 k변경가능
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00912786, 0.00853467, 0.00894475, 0.00893879, 0.0083797 ]), 'score_time': array([0.00134873, 0.00134277, 0.00124264, 0.00129223, 0.00119233]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
#검증점수를 평균하여 교차검증의 최종점수 획득
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
#교차검증시 훈련데이터를 섞으려면 사이킷런의 분할기를 이용
#회귀모델은 KFold분할기 사용, 분류모델은 StratifiedFold 사용
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
#훈련세트를 섞은 후 10-fold 교차검증을 수행
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
#하이퍼파라미터의 튜닝
#GridSearchCV 클래스: 하이퍼파라미터의 탐색과 교차검증을 수행
#결정트리에서 min_impurity_decrease 매개변수의 최적값을 탐색
#GridSearchCV 함수는 5폴드 * min_impurity_decrease 값 5 = 25개 모델을 테스트해서
#검증점수가 가장 높은 모델은 best_estimator에 저장
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
#최적의 모델로 평가
dt = gs.best_estimator_
print(gs.best_estimator_)
print(dt.score(train_input, train_target))

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
0.9615162593804117


In [ ]:
#그리드서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
#5번 교차 검증의 평균점수: cv_results_ 속성의 'mean_test_score' 키에 저장
print(gs.cv_results_)
print(gs.cv_results_['mean_test_score'])

{'mean_fit_time': array([0.01144814, 0.01020532, 0.00935969, 0.01318479, 0.01261292]), 'std_fit_time': array([0.00058949, 0.00098796, 0.00055568, 0.00662716, 0.0059963 ]), 'mean_score_time': array([0.00209446, 0.00198126, 0.00167255, 0.00217495, 0.00209403]), 'std_score_time': array([0.00041524, 0.00070604, 0.00014802, 0.00083202, 0.00059239]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value=1e+20), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161694, 0.8825794 ]), 'split3_test_score': arr

In [ ]:
#넘파이의 argmax 함수를 이용하여 최대값의 인덱스를 알아낸 후 gs 객체의 params 객체에서 확인
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
#그리드 서치를 이용하여 결정트리의 3가지 매개변수의 베스트 조합 결정
#1. min_impurity_decrease: 노드분할을 위한 불순도 감소 최소량 지정: 0.0001 ~ 0.0009까지 9개
#2. max_depth: 트리의 깊이: 5~20까지 15개
#3. min_samples_split: 노드분할을 위한 최소 샘플수 지정: 2~100까지 2씩 간격으로 10개
#모델의 수는 9 * 15 * 10 * 5(5폴드 교차검증) = 6750개

params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001), 'max_depth': range(5,20,1), 'min_samples_split':range(2,100,10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=14, min_impurity_decrease=np.float64(0.0004),
                       min_samples_split=12, random_state=42)


In [ ]:
from scipy.stats import uniform, randint
#randint(s,t) s-t-1 사이의 정수를 랜덤하게 뽑음
rgen = randint(0,10)
print(rgen.rvs(10)) #0~9 사이의 데이터가 10번 뽑힘

print(np.unique(rgen.rvs(1000), return_counts=True))

ugen = uniform(0,1)
print(ugen.rvs(10))

[0 9 8 3 4 5 3 2 8 9]
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 93, 105, 101, 102, 110,  92,  90, 118, 107,  82]))
[0.62488429 0.30262249 0.5903487  0.83063403 0.01159549 0.28437503
 0.35654678 0.6660918  0.18194623 0.51539724]
